<a href="https://colab.research.google.com/github/NikolasRR/selecao-de-mancal/blob/main/Sele%C3%A7%C3%A3o_de_Mancal_de_rolamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COLOQUE SEUS VALORES NA CÉLULA ABAIXO, EM SEGUIDA EXECUTE-A**

In [30]:
vel = 1350 #rpm
vidaEmHoras = 52560
a = 1/3 #Para mancal de esfera V = 1, para mancal de rolo V = 1.2
Fr = 698.8/1000 #Deixe o valor em kN
Fa = 199/1000 #Deixe o valor em kN
V = 1
FaVFr = Fa/(V*Fr)

**EXECUTE A CÉLULA ABAIXO PARA GERAR A TABELA DE VALORES**

In [31]:
#TABELA DE PARÂMETROS
tabela = [
    #[FaC0, e, X, Y]
    [0.014, 0.19, 0.56, 2.3],
    [0.021, 0.21, 0.56, 2.15],
    [0.028, 0.22, 0.56, 1.99],
    [0.042, 0.24, 0.56, 1.85],
    [0.056, 0.26, 0.56, 1.71],
    [0.070, 0.27, 0.56, 1.63],
    [0.084, 0.28, 0.56, 1.55],
    [0.110, 0.30, 0.56, 1.45],
    [0.17, 0.34, 0.56, 1.31],
    [0.28, 0.38, 0.56, 1.15],
    [0.42, 0.42, 0.56, 1.04],
    [0.56, 0.44, 0.56, 1],
]

def makeDict(array):
  return dict(
      FaC0 = array[0],
      e = array[1],
      X = array[2],
      Y = array[3]
  )

tabela = list(map(makeDict, tabela))

**EXECUTE A CÉLULA ABAIXO PARA DEFINIR AS FUNÇÕES NECESSÁRIAS**

In [36]:
def interpolar(previousX, previousY, currentY, futureX, futureY):
  return round(((((currentY - previousY)/(futureY - previousY))*(futureX - previousX)) + previousX), 2)

def calcularRazaoC10F():
  return ((vidaEmHoras*vel*60/(10e5))**a)

def calcularC10(F):
  return ((F * calcularRazaoC10F()))

def calcularFe(x, y):
  return x*Fr + y*Fa

def obterNovoC10():
  print('valor de C')
  C10 = float(input())
  return C10

def obterNovoC0():
  print('valor de C0')
  C0 = float(input())
  return C0

def obterValoresXeY(razaoFaC0):
  for index, linha in enumerate(tabela):
    if (linha['FaC0'] == razaoFaC0):
      return dict(X = linha['X'], Y = linha['Y'])
    elif ((linha['FaC0'] > razaoFaC0) and (index == 0)):
      if (FaVFr < linha['e']):
        return dict(X = 1, Y = 0)
      else:
        return dict(X = tabela[0]['X'], Y = tabela[0]['Y'])
    elif (linha['FaC0'] > razaoFaC0):
      e = interpolar(tabela[index - 1]['e'], tabela[index - 1]['FaC0'], razaoFaC0, linha['e'], linha['FaC0'])
      if (FaVFr >= e):
        return dict(X = 1, Y = 0)
      else:
        Y = interpolar(tabela[index - 1]['Y'], tabela[index - 1]['FaC0'], razaoFaC0, linha['Y'], linha['FaC0'])
        return dict(X = linha['X'], Y = Y)

  raise Exception("Valores fora da tabela")

def loopMancal():
  C10 = calcularC10(Fr)
  novoC10 = C10
  C0 = 0

  while (novoC10 >= C10):
    print("Escolha um mancal que possua C maior que: {:.2f} kN\nAgora sobre o mancal selecionado, informe:".format(C10))

    C10 = obterNovoC10()
    C0 = obterNovoC0()

    razaoFaC0 = Fa/C0
    print("razao Fa/C0 = {:.3f}".format(razaoFaC0))
    XeY = obterValoresXeY(razaoFaC0)

    Fe = calcularFe(XeY['X'], XeY['Y'])

    novoC10 = calcularC10(Fe)

    print("valor do novo C10: {:.2f}".format(novoC10))
    print("valor do velho C10: {:.2f}".format(C10))

  print("\nrolamento encontrado, valores em kN", "C10 =", C10, "e C0 =", C0)



print("Valores carregados")
print("Fa/VFr: {:.2f}".format(Fa/(V*Fr)))
print("valor de C10 inicial em kN: {:.2f}".format(calcularC10(Fr)))

Valores carregados
Fa/VFr: 0.28
valor de C10 inicial em kN: 11.33


**EXECUTE A CÉLULA ABAIXO PARA COMEÇAR O PROCESSO**

In [37]:
loopMancal()

Escolha um mancal que possua C maior que: 11.33 kN
Agora sobre o mancal selecionado, informe:
valor de C
11.4
valor de C0
8.15
razao Fa/C0 = 0.024
valor do novo C10: 11.33
valor do velho C10: 11.40

rolamento encontrado, valores em kN C10 = 11.4 e C0 = 8.15
